__Word Alignment Assignment__

Your task is to learn word alignments for the data provided with this Python Notebook. 

Start by running the 'train' function below and implementing the assertions which will fail. Then consider the following improvements to the baseline model:
* Is the TranslationModel parameterized efficiently?
* What form of PriorModel would help here? (Currently the PriorModel is uniform.)
* How could you use a Hidden Markov Model to model word alignment indices? (There's an implementation of simple HMM below to help you start.)
* How could you initialize more complex models from simpler ones?
* How could you model words that are not aligned to anything?

Grades will be assigned as follows*:

 AER below on blinds   |  Grade 
----------|-------------
 0.5 - 0.6 |   1 
 0.4 - 0.5 |   2 
 0.35 - 0.4 |  3    
 0.3 - 0.35 |  4    
 0.25 - 0.3 |  5   
 
You should save the notebook with the final scores for 'dev' and 'test' test sets.

*__Note__: Students who submitted a version of this assignment last year will have a 0.05 AER handicap, i.e to get a grade of 5, they will need to get an AER below 0.25.


In [0]:
# This cell contains the generative models that you may want to use for word alignment.
# Currently only the TranslationModel is at all functional.

import numpy as np
from collections import defaultdict


class TranslationModel:
    "Models conditional distribution over trg words given a src word."

    def __init__(self, src_corpus, trg_corpus):
        self._trg_given_src_probs = {}
        self._src_trg_counts = {}
        
    def get_params(self):
        return self._trg_given_src_probs

    def get_conditional_prob(self, src_token, trg_token):
        "Return the conditional probability of trg_token(f_j) given src_token(e_i)."
        if src_token not in self._trg_given_src_probs or trg_token not in self._trg_given_src_probs[src_token]:
            return 1.0
        return self._trg_given_src_probs[src_token][trg_token]

    def collect_statistics(self, src_tokens, trg_tokens, posterior_matrix):
        "Accumulate fractional alignment counts from posterior_matrix."
        assert len(posterior_matrix) == len(trg_tokens)
        for posterior in posterior_matrix:
            assert len(posterior) == len(src_tokens)
        for i, src in enumerate(src_tokens):
            self._src_trg_counts.setdefault(src, {})
            for j, trg in enumerate(trg_tokens):
                self._src_trg_counts[src][trg] = self._src_trg_counts[src].get(trg, 0) + posterior_matrix[j][i]

    def recompute_parameters(self):
        "Reestimate parameters from counts then reset counters"
        self._trg_given_src_probs = {}
        for src, trg_val in self._src_trg_counts.items():
            norm_const = sum(trg_val.values())
            self._trg_given_src_probs[src] = {trg: val / norm_const for trg, val in trg_val.items()}
        self._src_trg_counts = {}


class PriorModel:
    "Models the prior probability of an alignment given only the sentence lengths and token indices."

    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        self._distance_counts = {}
        self._distance_probs = {}

    def get_prior_prob(self, src_index, trg_index, src_length, trg_length):
        "Returns a uniform prior probability."
        if (src_length, trg_length) not in self._distance_probs:
            return 1.0 / src_length
        self._distance_probs[(src_length, trg_length)].setdefault(trg_index, [1.0 / src_length] * src_length)
        return self._distance_probs[(src_length, trg_length)][trg_index][src_index]

    def collect_statistics(self, src_length, trg_length, posterior_matrix):
        "Extract the necessary statistics from this matrix if needed."
        assert len(posterior_matrix) == trg_length
        for posterior in posterior_matrix:
            assert len(posterior) == src_length
        current = self._distance_counts.setdefault((src_length, trg_length), {})
        for j in range(trg_length):
            prob = current.setdefault(j, [0] * src_length)
            for i in range(src_length):
                prob[i] += posterior_matrix[j][i]

    def recompute_parameters(self):
        "Reestimate the parameters and reset counters."
        self._distance_probs = {}
        for src_length, trg_length in self._distance_counts.keys():
            self._distance_probs[(src_length, trg_length)] = {}
            current = self._distance_counts[(src_length, trg_length)]
            for j in range(trg_length):
                prob = current[j]
                norm_const = sum(prob)
                self._distance_probs[(src_length, trg_length)][j] = [pr / norm_const for pr in prob]
        self._distance_counts = {}
    

class TransitionModel:
    "Models the prior probability of an alignment conditioned on previous alignment."

    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        pass

    def get_parameters_for_sentence_pair(self, src_length):
        "Retrieve the parameters for this sentence pair: A[k, i] = p(a_{j} = i|a_{j-1} = k)"
        pass

    def collect_statistics(self, src_length, bigram_posteriors):
        "Extract statistics from the bigram posterior[i][j]: p(a_{t-1} = i, a_{t} = j| e, f)"
        pass
        
    def recompute_parameters(self):
        "Recompute the transition matrix"
        pass




In [0]:
# This cell contains the framework for training and evaluating a model using EM.

from utils import read_parallel_corpus, extract_test_set_alignments, score_alignments, write_aligned_corpus
import unidecode
from tqdm import tqdm

def infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model):
    "Compute the posterior probability p(a_j=i | f, e) for each target token f_j given e and f."
    # HINT: An HMM will require more complex statistics over the hidden alignments.
    results = []
    for trg_ind in range(len(trg_tokens)):
        mm = [
            translation_model.get_conditional_prob(src_tokens[src_ind], trg_tokens[trg_ind]) *
            prior_model.get_prior_prob(src_ind, trg_ind, len(src_tokens), len(trg_tokens))
            for src_ind in range(len(src_tokens))
        ]
        marginal = sum(mm)
        posterior = [mm[i] / marginal for i in range(len(mm))]
        results.append((marginal, posterior))
    posterior_matrix = [res[1] for res in results]
    sentence_marginal_log_likelihood = np.log([res[0] for res in results]).sum()
    return sentence_marginal_log_likelihood, posterior_matrix

def collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model):
    "E-step: infer posterior distribution over each sentence pair and collect statistics."
    corpus_marginal_log_likelihood = 0.0
    for i in range(len(src_corpus)):
        likelihood, posterior = infer_posteriors(src_corpus[i], trg_corpus[i],
                                                               prior_model, translation_model)
        prior_model.collect_statistics(len(src_corpus[i]), len(trg_corpus[i]), posterior)
        translation_model.collect_statistics(src_corpus[i], trg_corpus[i], posterior)
        corpus_marginal_log_likelihood += likelihood
    return corpus_marginal_log_likelihood

def estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations):
    "Estimate models iteratively using EM."
    for iteration in tqdm(range(num_iterations)):
        # E-step
        corpus_log_likelihood = collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model)
        # M-step
        prior_model.recompute_parameters()
        translation_model.recompute_parameters()
        if iteration > 0:
            print("corpus log likelihood: %1.3f" % corpus_log_likelihood)
    return prior_model, translation_model

def get_alignments_from_posterior(posteriors):
    "Returns the MAP alignment for each target word given the posteriors."
    # HINT: If you implement an HMM, you may want to implement a better algorithm here.
    alignments = {}
    for src_index, trg_index in enumerate(np.argmax(posteriors, 1)):
        if src_index not in alignments:
            alignments[src_index] = {}
        alignments[src_index][trg_index] = '*'
    return alignments

def align_corpus(src_corpus, trg_corpus, prior_model, translation_model):
    "Align each sentence pair in the corpus in turn."
    aligned_corpus = []
    for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
        _, posteriors = infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model)
        alignments = get_alignments_from_posterior(posteriors)
        aligned_corpus.append((src_tokens, trg_tokens, alignments))
    return aligned_corpus

def initialize_models(src_corpus, trg_corpus):
    prior_model = PriorModel(src_corpus, trg_corpus)
    translation_model = TranslationModel(src_corpus, trg_corpus)
    return prior_model, translation_model

def normalize(src_corpus, trg_corpus):
    for i in range(len(src_corpus)):
        for j in range(len(src_corpus[i])):
            src_corpus[i][j] = src_corpus[i][j].lower()
            src_corpus[i][j] = unidecode.unidecode(src_corpus[i][j])
            src_corpus[i][j] = src_corpus[i][j][:4]
    for i in range(len(trg_corpus)):
        for j in range(len(trg_corpus[i])):
            trg_corpus[i][j] = trg_corpus[i][j].lower()
            trg_corpus[i][j] = trg_corpus[i][j][:4]
            trg_corpus[i][j] = unidecode.unidecode(trg_corpus[i][j])
    return src_corpus, trg_corpus

def train(num_iterations):
    src_corpus, trg_corpus, _ = read_parallel_corpus('en-cs.all')
    src_corpus, trg_corpus = normalize(src_corpus, trg_corpus)
    prior_model, translation_model = initialize_models(src_corpus, trg_corpus)
    prior_model, translation_model = estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations)    
    aligned_corpus = align_corpus(src_corpus, trg_corpus, prior_model, translation_model)
    return aligned_corpus, extract_test_set_alignments(aligned_corpus)

def evaluate(candidate_alignments):
    src_dev, trg_dev, wa_dev = read_parallel_corpus('en-cs-wa.dev', has_alignments=True)
    src_test, trg_test, wa_test = read_parallel_corpus('en-cs-wa.test', has_alignments=True)
    print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_dev, candidate_alignments['dev']))
    print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_test, candidate_alignments['test']))            

In [78]:
aligned_corpus, test_alignments = train(10)
evaluate(test_alignments)

 20%|██        | 2/10 [00:52<03:08, 23.51s/it]

corpus log likelihood: -1406258.593


 30%|███       | 3/10 [01:24<03:03, 26.16s/it]

corpus log likelihood: -1114204.843


 40%|████      | 4/10 [01:57<02:48, 28.03s/it]

...

 90%|█████████ | 9/10 [04:41<00:31, 31.91s/it]

corpus log likelihood: -692657.969


100%|██████████| 10/10 [05:13<00:00, 32.08s/it]

corpus log likelihood: -689685.077


recall 0.648; precision 0.583; aer 0.387
recall 0.649; precision 0.577; aer 0.390


In [0]:
# To visualize aligned corpus:
# 1. call write_aligned_corpus(aligned_corpus, 'out')
# 2. run python corpus_browser.py en-cs-wa.out (in working directory)

In [0]:
# Discrete HMM with scaling. You may want to use this if you decide to implement an HMM.
# The parameters for this HMM will still need to be provided by the models above.

def forward(pi, A, O):
    S, T = O.shape
    alpha = np.zeros((S, T))
    scaling_factors = np.zeros(T)
    
    # base case
    alpha[:, 0] = pi * O[:, 0]
    scaling_factors[0] = np.sum(alpha[:, 0])
    alpha[:, 0] /= scaling_factors[0] 
    
    # recursive case
    for t in range(1, T):
        alpha[:, t] = np.dot(alpha[:, t-1], A[:, :]) * O[:, t]

        # Normalize at each step to prevent underflow.
        scaling_factors[t] = np.sum(alpha[:, t])
        alpha[:, t] /= scaling_factors[t]

    return (alpha, scaling_factors)

def backward(pi, A, O, forward_scaling_factors):
    S, T = O.shape
    beta = np.zeros((S, T))

    # base case
    beta[:, T-1] = 1 / forward_scaling_factors[T-1]
    
    # recursive case
    for t in range(T-2, -1, -1):
        beta[:, t] = np.sum(beta[:, t+1] * A[:, :] * O[:, t+1], 1) / forward_scaling_factors[t]

    return beta

def forward_backward(pi, A, O):
    alpha, forward_scaling_factors = forward(pi, A, O)
    beta = backward(pi, A, O, forward_scaling_factors)
    return alpha, beta, np.sum(np.log(forward_scaling_factors))
